In [1]:
import sys
print(sys.version)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]


In [1]:
!nvidia-smi

Tue Oct  8 10:40:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import statistics
import json
import pandas as pd
import re
import nltk
import numpy as np
import scipy.stats as stats
nltk.download('punkt')

import os
os.listdir('/kaggle/input/')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['indian-supreme-court-judgements', 'indian-supreme-court-judgement-dataset']

In [2]:
file_path = '/kaggle/input/indian-supreme-court-judgement-dataset/used_dataset.json'

# Read the JSON file
with open(file_path, 'r') as f:
    dataset = json.load(f)

# #Check one document
# print(dictionaries[0])

In [3]:
!pip install rouge_score sacrebleu torch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bd157060ac40912790a553d37f3ff8267d3cba8ee7bb1c87ae6bd1cd6623aa6f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
from rouge_score import rouge_scorer
import sacrebleu
import torch
print(torch.__version__)
print(torch.version.cuda)

2.4.0
12.3


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer1 = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
model1 = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus").to(device)

tokenizer2 = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")
model2 = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384").to(device)

tokenizer3 = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model3 = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6").to(device)

tokenizer4 = AutoTokenizer.from_pretrained("law-ai/InCaseLawBERT")
model4 = AutoModel.from_pretrained("law-ai/InCaseLawBERT").to(dev)

tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/343 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/534M [00:00<?, ?B/s]

In [7]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def preprocessing(text):
    text= text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Rejoin tokens back into a string
    return ' '.join(filtered_tokens)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def tokenize_sentences(summary):
    return [nltk.word_tokenize(sentence) for sentence in summary]

def pad_sentences(reference, predicted):
    max_len = max(len(reference), len(predicted))
    
    # Padding reference and predicted summaries
    padded_reference = reference + [["<PAD>"]] * (max_len - len(reference))
    padded_predicted = predicted + [["<PAD>"]] * (max_len - len(predicted))
    
    return padded_reference, padded_predicted

def flatten_sentences(sentences):
    return [word for sentence in sentences for word in sentence]

from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_scores(reference, predicted):
    # Flatten the sentence lists
    flat_reference = flatten_sentences(reference)
    flat_predicted = flatten_sentences(predicted)
    
    if len(flat_reference) != len(flat_predicted):
        # Padding again if necessary
        if len(flat_reference) > len(flat_predicted):
            flat_predicted += ['<PAD>'] * (len(flat_reference) - len(flat_predicted))
        else:
            flat_reference += ['<PAD>'] * (len(flat_predicted) - len(flat_reference))
    
    # Convert words into binary classes for precision/recall/f1 (1 if word matches, 0 otherwise)
    reference_binary = [1 if word in flat_predicted else 0 for word in flat_reference]
    predicted_binary = [1 if word in flat_reference else 0 for word in flat_predicted]
    
    # Calculate precision, recall, and F1
    precision = precision_score(reference_binary, predicted_binary, average='weighted', zero_division=0)
    recall = recall_score(reference_binary, predicted_binary, average='weighted', zero_division=0)
    f1 = f1_score(reference_binary, predicted_binary, average='weighted', zero_division=0)
    
    return precision, recall, f1


In [9]:
def generate_summary(text, tokenizer, model):
    input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True).to(device)
    summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=300,
                                  max_length=3000,
                                  early_stopping=True)
    summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
    # print(summary)
    return summary

def evaluate_summary(generated_summary, reference_summary, ref):
    
    reference = [nltk.word_tokenize(" ".join(reference_summary))]
    hypothesis =[generated_summary]
    ref1 = [nltk.word_tokenize(sentence) for sentence in ref]

    # BLEU score using sacrebleu
    bleu_score = sacrebleu.corpus_bleu(hypothesis, ref1).score

    # ROUGE score calculation
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(reference_summary, generated_summary)
    
    # Tokenize the sentences
    tokenized_reference = tokenize_sentences(reference_summary)
    tokenized_predicted = tokenize_sentences(nltk.sent_tokenize(generated_summary))

    # Pad the sentences if necessary
    padded_reference, padded_predicted = pad_sentences(tokenized_reference, tokenized_predicted)

    # Calculate Precision, Recall, and F1
    precision, recall, f1 = calculate_scores(padded_reference, padded_predicted)

#     precision, recall, f1 = precision_recall_f1(" ".join(reference_summary), generated_summary)

    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Checking direct abstractive methods

In [15]:
def evaluate_dataset_app(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        reference_summary = " ".join(doc['headnote_sent'])
        judgement =   [preprocessing(i) for i in doc['judgement_sent']]
        judgement_text = " ".join(judgement)
        ref = doc['headnote_sent']
        
        
        generated_summary = generate_summary(judgement_text, tokenizer, model)

        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [ ]:
avg_scores_app_1= evaluate_dataset_app(dataset, tokenizer1, model1)
print(avg_scores_app_1)

In [12]:
avg_scores_app_2= evaluate_dataset_app(dataset, tokenizer2, model2)
print(avg_scores_app_2)

1 

Input ids are automatically padded from 372 to 1024 to be a multiple of `config.attention_window`: 1024


2 3 4 5 6 

Input ids are automatically padded from 765 to 1024 to be a multiple of `config.attention_window`: 1024


7 

...

369 370 371 372 373 

Input ids are automatically padded from 527 to 1024 to be a multiple of `config.attention_window`: 1024


374 

Input ids are automatically padded from 905 to 1024 to be a multiple of `config.attention_window`: 1024


375 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 
{'bleu': 0.11498283224489805, 'rouge1': 0.29448841162708245, 'rouge2': 0.05706555935478016, 'rougeL': 0.15256813196176203, 'precision': 0.6988614924652756, 'recall': 0.4039308615275846, 'f1': 0.30593763631902215}


In [11]:
avg_scores_app_3= evaluate_dataset_app(dataset, tokenizer3, model3)
print(avg_scores_app_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

# MODEL 1 - Legal-BERT

In [9]:
from transformers import BertTokenizer, BertModel
import torch

# Load the Legal-BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')
model = BertModel.from_pretrained('nlpaueb/legal-bert-base-uncased').to(device)

def get_legalbert_embeddings(sentences):
    embeddings = []
    
    for sentence in sentences:
        
        # Tokenize the sentence and convert to input IDs and attention masks
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        with torch.no_grad():
            outputs = model(**inputs)
        
        # The embeddings are the output of the last hidden state
        # We take the mean of the token embeddings to get a sentence embedding
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).cpu().squeeze()
        embeddings.append(sentence_embedding)

    return torch.stack(embeddings)
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity_scores(embeddings, threshold=0.4):
    sentence_count = len(embeddings)
    counts = [0] * sentence_count  # Initialize a count for each sentence

    for i in range(sentence_count):
        for j in range(sentence_count):
            if i != j:
                similarity = cosine_similarity(
                    embeddings[i].unsqueeze(0), embeddings[j].unsqueeze(0)
                )[0][0]
                if similarity > threshold:
                    counts[i] += 1
    return counts
def select_top_sentences(sentences, counts, top_percent=0.4):
    # Zip sentences with their counts and sort based on counts
    sorted_results = sorted(zip(sentences, counts), key=lambda x: x[1], reverse=True)
    num_sentences = int(top_percent * len(sorted_results))  # Top 40%
    top_sentences = [item[0] for item in sorted_results[:num_sentences]]  # Select sentences
    return top_sentences
def generate_summary_from_top_sentences(top_sentences):
    return ' '.join(top_sentences)


def evaluate_summary(generated_summary, reference_summary, ref):
    # Tokenization for BLEU score calculation
    reference = [nltk.word_tokenize(reference_summary)]
    hypothesis = nltk.sent_tokenize(generated_summary)
    ref_tokenized = [nltk.word_tokenize(sentence) for sentence in ref]

    # BLEU score using sacrebleu
    bleu_score = sacrebleu.corpus_bleu(hypothesis, ref_tokenized).score

    # ROUGE score calculation
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(" ".join(reference_summary), " ".join(generated_summary))
    
    #     precision, recall, f1 = precision_recall_f1(" ".join(reference_summary), generated_summary)
    # Tokenize the sentences
    tokenized_reference = tokenize_sentences(reference_summary)
    tokenized_predicted = tokenize_sentences(nltk.sent_tokenize(generated_summary))

    # Pad the sentences if necessary
    padded_reference, padded_predicted = pad_sentences(tokenized_reference, tokenized_predicted)

    # Calculate Precision, Recall, and F1
    precision, recall, f1 = calculate_scores(padded_reference, padded_predicted)

    # You can remove or replace precision/recall/F1 with more appropriate metrics if needed
    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "precision":precision,
        "recall":recall,
        "f1":f1
    }


def evaluate_dataset_app1(dataset, tokenizer, model):
    count=0
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
      }

    for doc in dataset:
        
        judgement_text = [ preprocessing(i) for i in doc['judgement_sent']]
        reference_summary = (doc['headnotes_sent'])
        ref = doc['headnotes_sent']

        
        sentence_embeddings = get_legalbert_embeddings(judgement_text)
    
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)
    
        
        top_sentences = select_top_sentences(judgement_text, counts, top_percent=0.4)
    
        
        filtered_summary = generate_summary_from_top_sentences(top_sentences)
        generated_summary = generate_summary(filtered_summary, tokenizer, model)
    
        
        scores = evaluate_summary((generated_summary), " ".join(reference_summary), ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count+=1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [14]:
avg_scores_app1_1= evaluate_dataset_app1(dataset, tokenizer1, model1)
print(avg_scores_app1_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [10]:
avg_scores_app1_2= evaluate_dataset_app1(dataset, tokenizer2, model2)
print(avg_scores_app1_2)

1 2 

Input ids are automatically padded from 119 to 1024 to be a multiple of `config.attention_window`: 1024


3 

Input ids are automatically padded from 353 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

396 

Input ids are automatically padded from 861 to 1024 to be a multiple of `config.attention_window`: 1024


397 398 399 

Input ids are automatically padded from 632 to 1024 to be a multiple of `config.attention_window`: 1024


400 
{'bleu': 0.2497404295645549, 'rouge1': 0.6426056064356997, 'rouge2': 0.5514431256206693, 'rougeL': 0.37073537440073406, 'precision': 0.7023600939620879, 'recall': 0.3908989961775458, 'f1': 0.2902722423268203}


In [11]:
avg_scores_app1_3= evaluate_dataset_app1(dataset, tokenizer3, model3)
print(avg_scores_app1_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

# MODEL 2- TEXTRANK with TF-IDF

In [20]:
!pip install networkx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

def compute_sentence_similarity(sentences):
    # Use TF-IDF Vectorizer to convert sentences to vectors
    vectorizer = TfidfVectorizer().fit_transform(sentences)
    # Compute cosine similarity between sentence vectors
    similarity_matrix = cosine_similarity(vectorizer)
    return similarity_matrix


def rank_sentences_with_textrank(sentences, similarity_matrix):
    # Create a graph where each node is a sentence
    nx_graph = nx.from_numpy_array(similarity_matrix)

    # Apply the TextRank (PageRank) algorithm on the graph
    scores = nx.pagerank(nx_graph)

    # Rank sentences based on their TextRank score
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    return ranked_sentences


def select_top_sentences_textrank(ranked_sentences, top_percent=0.4):
    # Select the top X% of sentences
    num_sentences = int(top_percent * len(ranked_sentences))
    top_sentences = [sent[1] for sent in ranked_sentences[:num_sentences]]
    return top_sentences

def evaluate_dataset_app2(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        reference_summary = " ".join(doc['headnotes_sent'])
        judgement =   [preprocessing(i) for i in doc['judgement_sent']]
        ref = doc['headnotes_sent']

        
        similarity_matrix = compute_sentence_similarity(judgement)

        
        ranked_sentences = rank_sentences_with_textrank(judgement, similarity_matrix)

        
        top_sentences = select_top_sentences_textrank(ranked_sentences, top_percent=0.4)

        
        generated_summary = generate_summary(" ".join(top_sentences), tokenizer, model)

        
        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
avg_scores_app2_1= evaluate_dataset_app2(dataset, tokenizer1, model1)
print(avg_scores_app2_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [17]:
avg_scores_app2_2= evaluate_dataset_app2(dataset, tokenizer2, model2)
print(avg_scores_app2_2)

1 

Input ids are automatically padded from 187 to 1024 to be a multiple of `config.attention_window`: 1024


2 

Input ids are automatically padded from 804 to 1024 to be a multiple of `config.attention_window`: 1024


3 

...

391 

Input ids are automatically padded from 895 to 1024 to be a multiple of `config.attention_window`: 1024


392 393 394 395 396 397 398 

Input ids are automatically padded from 778 to 1024 to be a multiple of `config.attention_window`: 1024


399 400 
{'bleu': 0.12216438869202265, 'rouge1': 0.28931277832726127, 'rouge2': 0.05569109282517037, 'rougeL': 0.15051339647392256, 'precision': 0.7001716571777682, 'recall': 0.3970917106919879, 'f1': 0.29728355723903926}


In [18]:
avg_scores_app2_3= evaluate_dataset_app2(dataset, tokenizer3, model3)
print(avg_scores_app2_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

# MODEL 3  - LSA

In [16]:
def compute_similarity_scores(embeddings, threshold=0.4):
    sentence_count = len(embeddings)
    counts = [0] * sentence_count  # Initialize a count for each sentence

    for i in range(sentence_count):
        for j in range(sentence_count):
            if i != j:
                similarity = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1))[0][0]
                if similarity > threshold:
                    counts[i] += 1
    return counts

def select_top_sentences(sentences, counts, top_percent=0.4):
    sorted_results = sorted(zip(sentences, counts), key=lambda x: x[1], reverse=True)
    num_sentences = int(top_percent * len(sorted_results))  # Top 40%
    top_sentences = [item[0] for item in sorted_results[:num_sentences]]  # Select sentences
    return top_sentences

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

def compute_sentence_embeddings_lsa(sentences, n_components=100):
 
  vectorizer = TfidfVectorizer(max_features=5000)  
 
  tfidf_matrix = vectorizer.fit_transform([preprocessing(i) for i in sentences])

  svd_model = TruncatedSVD(n_components=n_components, random_state=42)
  sentence_embeddings = svd_model.fit_transform(tfidf_matrix)

  return sentence_embeddings

In [25]:
def evaluate_dataset_app3(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        
        reference_summary = " ".join(doc['headnotes_sent'])
        judgement =   [preprocessing(i) for i in doc['judgement_sent']]
        ref = doc['headnotes_sent']

        
        #sentence_embeddings = get_sentence_embeddings(judgement)
        sentence_embeddings = compute_sentence_embeddings_lsa(judgement)
        
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)

        
        top_sentences = select_top_sentences(judgement, counts, top_percent=0.4)

       
        generated_summary = generate_summary(" ".join(top_sentences), tokenizer, model)

        
        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [25]:
avg_scores_app3_1= evaluate_dataset_app3(dataset, tokenizer1, model1)
print(avg_scores_app3_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [26]:
avg_scores_app3_3= evaluate_dataset_app3(dataset, tokenizer3, model3)
print(avg_scores_app3_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [28]:
avg_scores_app3_2= evaluate_dataset_app3(dataset, tokenizer2, model2)
print(avg_scores_app3_2)

1 

Input ids are automatically padded from 156 to 1024 to be a multiple of `config.attention_window`: 1024


2 3 

Input ids are automatically padded from 727 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

393 394 395 396 

Input ids are automatically padded from 930 to 1024 to be a multiple of `config.attention_window`: 1024


397 398 399 

Input ids are automatically padded from 601 to 1024 to be a multiple of `config.attention_window`: 1024


400 
{'bleu': 0.12083071935646414, 'rouge1': 0.28983924596805016, 'rouge2': 0.05599103360952234, 'rougeL': 0.15175355982060382, 'precision': 0.7024397687099693, 'recall': 0.3935947464621091, 'f1': 0.2939206088342424}


# MODEL 4 - InCaseLawBERT

In [24]:
def get_sentence_embeddings_in(sentences):
    embeddings = []
    for sentence in sentences:
        inputs = tokenizer4(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        with torch.no_grad():
            output = model4(**inputs)
        # Use the mean of the last hidden state as the sentence embedding
        embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(embedding)
    return embeddings

In [25]:
def evaluate_dataset_app4(dataset, tokenizer, model):
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
    }
    count=0

    for doc in dataset:
        reference_summary = " ".join(doc['headnotes_sent'])
        judgement =   [preprocessing(i) for i in doc['judgement_sent']]
        ref = doc['headnotes_sent']

        
        sentence_embeddings = get_sentence_embeddings_in(judgement)
        
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)

        
        top_sentences = select_top_sentences(judgement, counts, top_percent=0.4)

       
        generated_summary = generate_summary(" ".join(top_sentences), tokenizer, model)

        
        scores = evaluate_summary(generated_summary, reference_summary, ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count += 1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

In [26]:
avg_scores_app4_1= evaluate_dataset_app4(dataset, tokenizer1, model1)
print(avg_scores_app4_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [27]:
avg_scores_app4_3= evaluate_dataset_app4(dataset, tokenizer3, model3)
print(avg_scores_app4_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [28]:
avg_scores_app4_2= evaluate_dataset_app4(dataset, tokenizer2, model2)
print(avg_scores_app4_2)

1 2 

Input ids are automatically padded from 172 to 1024 to be a multiple of `config.attention_window`: 1024


3 

Input ids are automatically padded from 393 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

394 395 

Input ids are automatically padded from 1006 to 1024 to be a multiple of `config.attention_window`: 1024


396 397 398 399 

Input ids are automatically padded from 719 to 1024 to be a multiple of `config.attention_window`: 1024


400 
{'bleu': 0.12140598629995811, 'rouge1': 0.29111020024074274, 'rouge2': 0.05688017713193659, 'rougeL': 0.15230372226907712, 'precision': 0.7024690075471638, 'recall': 0.39298890187408664, 'f1': 0.2930693604697324}


# MODEL 5 - Legal RoBERTa

In [10]:
tokenizer5 = AutoTokenizer.from_pretrained("saibo/legal-roberta-base")
model5 = AutoModel.from_pretrained("saibo/legal-roberta-base").to(device)

def get_legalroberta_embeddings(sentences):
    embeddings = []
    
    for sentence in sentences:
        # Tokenize the sentence and convert to input IDs and attention masks
        inputs = tokenizer5(sentence, return_tensors='pt', padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()} 
        with torch.no_grad():
            outputs = model5(**inputs)
        
        # The embeddings are the output of the last hidden state
        # We take the mean of the token embeddings to get a sentence embedding
        sentence_embedding = outputs.last_hidden_state.mean(dim=1).cpu().squeeze()
        embeddings.append(sentence_embedding)

    return torch.stack(embeddings)
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity_scores(embeddings, threshold=0.4):
    sentence_count = len(embeddings)
    counts = [0] * sentence_count  # Initialize a count for each sentence

    for i in range(sentence_count):
        for j in range(sentence_count):
            if i != j:
                similarity = cosine_similarity(
                    embeddings[i].unsqueeze(0), embeddings[j].unsqueeze(0)
                )[0][0]
                if similarity > threshold:
                    counts[i] += 1
    return counts
def select_top_sentences(sentences, counts, top_percent=0.4):
    # Zip sentences with their counts and sort based on counts
    sorted_results = sorted(zip(sentences, counts), key=lambda x: x[1], reverse=True)
    num_sentences = int(top_percent * len(sorted_results))  # Top 40%
    top_sentences = [item[0] for item in sorted_results[:num_sentences]]  # Select sentences
    return top_sentences
def generate_summary_from_top_sentences(top_sentences):
    return ' '.join(top_sentences)


def evaluate_summary(generated_summary, reference_summary, ref):
    # Tokenization for BLEU score calculation
    reference = [nltk.word_tokenize(reference_summary)]
    hypothesis = nltk.sent_tokenize(generated_summary)
    ref_tokenized = [nltk.word_tokenize(sentence) for sentence in ref]

    # BLEU score using sacrebleu
    bleu_score = sacrebleu.corpus_bleu(hypothesis, ref_tokenized).score

    # ROUGE score calculation
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = rouge.score(" ".join(reference_summary), " ".join(generated_summary))
    
    # Tokenize the sentences
    tokenized_reference = tokenize_sentences(reference_summary)
    tokenized_predicted = tokenize_sentences(nltk.sent_tokenize(generated_summary))

    # Pad the sentences if necessary
    padded_reference, padded_predicted = pad_sentences(tokenized_reference, tokenized_predicted)

    # Calculate Precision, Recall, and F1
    precision, recall, f1 = calculate_scores(padded_reference, padded_predicted)


    # You can remove or replace precision/recall/F1 with more appropriate metrics if needed
    return {
        "bleu": bleu_score,
        "rouge1": rouge_scores['rouge1'].fmeasure,
        "rouge2": rouge_scores['rouge2'].fmeasure,
        "rougeL": rouge_scores['rougeL'].fmeasure,
        "precision":precision,
        "recall":recall,
        "f1":f1
    }


def evaluate_dataset_app5(dataset, tokenizer, model):
    count=0
    total_scores = {
      "bleu": 0,
      "rouge1": 0,
      "rouge2": 0,
      "rougeL": 0,
      "precision": 0,
      "recall": 0,
      "f1": 0
      }

    for doc in dataset:
        
        judgement_text = [preprocessing(i) for i in doc['judgement_sent']]
        reference_summary = (doc['headnote_sent'])
        ref = doc['headnote_sent']

        
        sentence_embeddings = get_legalroberta_embeddings(judgement_text)
    
        
        counts = compute_similarity_scores(sentence_embeddings, threshold=0.4)
    
        
        top_sentences = select_top_sentences(judgement_text, counts, top_percent=0.4)
    
        
        filtered_summary = generate_summary_from_top_sentences(top_sentences)
        generated_summary = generate_summary(filtered_summary, tokenizer, model)
    
        
        scores = evaluate_summary((generated_summary), " ".join(reference_summary), ref)

        # Accumulate the scores
        for key in total_scores:
            total_scores[key] += scores[key]
        count+=1
        print(count, end=" ")
    print()

    # Calculate average for each metric
    num_docs = len(dataset)
    avg_scores = {key: total_scores[key] / num_docs for key in total_scores}
    return avg_scores

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at saibo/legal-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
avg_scores_app5_1= evaluate_dataset_app5(dataset, tokenizer1, model1)
print(avg_scores_app5_1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [13]:
avg_scores_app5_3= evaluate_dataset_app5(dataset, tokenizer3, model3)
print(avg_scores_app5_3)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [11]:
avg_scores_app5_2= evaluate_dataset_app5(dataset, tokenizer2, model2)
print(avg_scores_app5_2)

1 2 

Input ids are automatically padded from 172 to 1024 to be a multiple of `config.attention_window`: 1024


3 

Input ids are automatically padded from 353 to 1024 to be a multiple of `config.attention_window`: 1024


4 

...

394 395 

Input ids are automatically padded from 917 to 1024 to be a multiple of `config.attention_window`: 1024


396 397 398 399 

Input ids are automatically padded from 632 to 1024 to be a multiple of `config.attention_window`: 1024


400 
{'bleu': 0.24631307285922013, 'rouge1': 0.6549628776334362, 'rouge2': 0.5597809443964391, 'rougeL': 0.3733296495013487, 'precision': 0.699273310734907, 'recall': 0.39691449000991497, 'f1': 0.2972101083598877}
